<a href="https://colab.research.google.com/github/milleniana/OrionTech-PackEco/blob/masters/Image_Classification_PackEco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

In [ ]:
import os
import zipfile
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.optimizers import Adam
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

# Acquiring Data

In [ ]:
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download kmader/food41

In [ ]:
!chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/ && echo 'Done'

# Generate Dataset

In [ ]:
#unzip dataset
local_zip = '/content/food41.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
print(len(os.listdir('/content/images/chicken_curry')))
print(len(os.listdir('/content/images/hot_and_sour_soup/')))
print(len(os.listdir('/content/images/ramen/')))
print(len(os.listdir('/content/images/donuts/')))
print(len(os.listdir('/content/images/fried_rice/')))
print(len(os.listdir('/content/images/hamburger/')))

In [ ]:
# make new directory for necessary dataset

os.mkdir('/tmp/makanan-basah-vs-makanan-kering')
os.mkdir('/tmp/makanan-basah-vs-makanan-kering/basah')
os.mkdir('/tmp/makanan-basah-vs-makanan-kering/kering')

In [ ]:
# move necessary data to makanan basah directory
source_b1 = r'/content/images/chicken_curry'
source_b2 = r'/content/images/hot_and_sour_soup'
source_b3  = r'/content/images/ramen'
destination_b = r'/tmp/makanan-basah-vs-makanan-kering/basah'

def path(SOURCE, DESTINATION):
  files = os.listdir(SOURCE)
  for file in files:
    new_path = shutil.move(f"{SOURCE}/{file}", DESTINATION)
    print(new_path)

path(source_b1, destination_b)
path(source_b2, destination_b)
path(source_b3, destination_b)

In [ ]:
# move necessary files to makanan kering directory
source_k1 = r'/content/images/donuts'
source_k2 = r'/content/images/fried_rice'
source_k3  = r'/content/images/hamburger'
destination_k = r'/tmp/makanan-basah-vs-makanan-kering/kering'

def path(SOURCE, DESTINATION):
  files = os.listdir(SOURCE)
  for file in files:
    new_path = shutil.move(f"{SOURCE}/{file}", DESTINATION)
    print(new_path)

path(source_k1, destination_k)
path(source_k2, destination_k)
path(source_k3, destination_k)

# Visualize Dataset

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in destination_k.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

# Data Preprocessing

In [ ]:
dataset_folder = '/tmp/makanan-basah-vs-makanan-kering'
image_size = (224, 224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_folder,
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=1002,
    validation_split=0.2,
    subset="training")

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_folder,
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    seed=1002,
    image_size=image_size,
    validation_split=0.2,
    subset="validation")

# Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
     layers.experimental.preprocessing.Rescaling(scale=1./255),
     layers.experimental.preprocessing.Normalization(),
     layers.experimental.preprocessing.RandomFlip("horizontal"),
     layers.experimental.preprocessing.RandomRotation(0.4),
     layers.experimental.preprocessing.RandomZoom(height_factor=(0.1, 0.3), width_factor=(0.1, 0.3)),
     layers.experimental.preprocessing.RandomHeight(factor=(0.1, 0.2)),
     layers.experimental.preprocessing.RandomWidth(factor=(0.1, 0.2))
    ]
)

In [ ]:
augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))


In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy())
        plt.axis("off")

# Configure Dataset for Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Transfer Learning

## Build The Model

In [ ]:
base_model = MobileNetV2(input_shape=(224, 224, 3), input_tensor=None, weights='imagenet', include_top=False)

base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = data_augmentation(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [ ]:
base_model.trainable = True
model.summary()

## Training and Validating

In [ ]:
model.compile(optimizer = keras.optimizers.Adam(1e-5),
              loss = keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,
                    epochs = 10,
                    validation_data = val_ds)

## Model Evaluation

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#Plot training accuracy and validation accuracy per epoch

plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.figure()

In [ ]:
#Plot training loss and validation loss per epoch

plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training Loss and Validation Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.figure()

## Predicting Image

In [ ]:
img = keras.preprocessing.image.load_img(
    "/content/images/miso_soup/1049083.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
imgplot = plt.imshow(img)
plt.axis('off')

predictions = model.predict(img_array)
score = predictions[0]

# convert score to the absolute value
b = abs(1-score)
k = abs(score)

# predict result
if b > k:
  print("Makanan Basah")
else:
  print("Makanan Kering")